### Sleep fraction during 5 hours following MAZE

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from neuropy import plotting
from neuropy.analyses import Pf1D
from neuropy.analyses import Decode1d
import subjects

sessions = subjects.sd.brainstates_sess + subjects.nsd.brainstates_sess

In [5]:
sleep_df = pd.DataFrame()
for sub, sess in enumerate(sessions):
    post= sess.paradigm["post"].flatten()
    period = [post[0],post[0] + 5 * 3600]
    prop = sess.brainstates.get_proportion_by_label(period[0], period[1])
    if "nan" in prop.keys():
        del prop["nan"]

    sleep_df = sleep_df.append(
        pd.DataFrame(
            {
                "prop": list(prop.values()),
                "state": list(prop.keys()),
                "grp":sess.tag,
            }
        ),
        ignore_index=True,
    )

subjects.GroupData().save(sleep_df,'brainstates_proportion')